In [1]:
!pip install unidecode

In [2]:
import pandas as pd
from unidecode import unidecode

In [3]:
df = pd.read_csv("movies_info.csv")

In [4]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [5]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [6]:
df.loc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [7]:
eval(df.loc[0]['genres'])

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [8]:
def clean_non_ascii_text(t):
    return unidecode(str(t))

In [9]:
unidecode("WALL��E")

'WALLE'

In [10]:
movie_details = df[['id', 'budget', 'original_language', 'title', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'tagline', 'vote_average', 'vote_count']].copy()

In [11]:
movie_details['title'] = movie_details['title'].apply(clean_non_ascii_text)
movie_details['overview'] = movie_details['overview'].apply(clean_non_ascii_text)
movie_details['tagline'] = movie_details['tagline'].apply(clean_non_ascii_text)

In [12]:
movie_details.head(2)

,id,budget,original_language,title,overview,popularity,release_date,revenue,runtime,tagline,vote_average,vote_count
0,19995,237000000,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800
1,285,300000000,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500


In [13]:
!mkdir csv_for_db

A subdirectory or file csv_for_db already exists.


In [14]:
movie_details.rename(columns = {'id':'movieid', }, inplace = True)

In [15]:
movie_details.to_csv('csv_for_db/movie_details_al.csv', index=False)

In [16]:
movie_details.shape

(4803, 12)

We plan to create the following tables:

1. movie_details
2. genre_details
3. movie_genre
4. production_company_details
5. movie_production
....

### Format json data

In [17]:
for row in df.iterrows():
    print(row[1])
    print(row[1]['genres'])
    break

budget                                                          237000000
genres                  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
homepage                                      http://www.avatarmovie.com/
id                                                                  19995
keywords                [{"id": 1463, "name": "culture clash"}, {"id":...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [{"name": "Ingenious Film Partners", "id": 289...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                               

In [18]:
movie_genre_tuples = []

for row in df.iterrows():
    mg = eval(row[1]['genres'])
    movieid = row[1]['id']
    
    for ele in mg:
        movie_genre_tuples.append((movieid, ele['id']))

print(len(movie_genre_tuples))
print(movie_genre_tuples[0:2])

12160
[(19995, 28), (19995, 12)]


# Steps for data insertion

1. Create the tables: 

create table moviedetails (movieid varchar (20), .....)

create table genredetails (genre_id varchar (20), genre_name ...)

create table moviegenre (movieid int (20), genreid int ...)

2. Import the above created csv into the table either using a python script, or using a tool such as mysqlworkbench (https://downloads.mysql.com/archives/workbench/).

------

drop table moviegenre;

CREATE TABLE moviegenre (movieid int(6) NOT NULL, genreid int(5) NOT NULL,  primary key (movieid, genreid),  foreign key (movieid) references moviedetails(movieid), foreign key (genreid) references genredetails(genre_id));

2.a. Using Python

2.b. Using MySQLWOrkbench

In [19]:
with open('csv_for_db/moviegenre.csv', 'w') as f:
    f.write('movieid,genreid\n')
    for _, ele in enumerate(movie_genre_tuples):
        f.write('{},{}\n'.format(ele[0], ele[1]))
        
        if _ > 10:
            break

In [20]:
!head csv_for_db/moviegenre.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
!wc -l csv_for_db/moviegenre.csv

'wc' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
all_genres = {}

for r in df.iterrows():
    gen_list = r[1]['genres']
    gl = eval(gen_list)
    for gen_tup in gl:
        all_genres[gen_tup["id"]] = unidecode(str(gen_tup["name"].replace(",", "")))

In [23]:
all_kws = {}

for r in df.iterrows():
    gen_list = r[1]['keywords']
    gl = eval(gen_list)
    for gen_tup in gl:
        all_kws[gen_tup["id"]] = unidecode(str(gen_tup["name"].replace(",", "")))

In [24]:
all_production_countries = {}

for r in df.iterrows():
    gen_list = r[1]['production_countries']
    gl = eval(gen_list)
    for gen_tup in gl:
        all_production_countries[gen_tup["iso_3166_1"]] = unidecode(str(gen_tup["name"].replace(",", "")))

In [25]:
all_production_companies = {}

for r in df.iterrows():
    gen_list = r[1]['production_companies']
    gl = eval(gen_list)
    for gen_tup in gl:
        all_production_companies[gen_tup["id"]] = unidecode(str(gen_tup["name"].replace(",", "")))

In [26]:
data_dicts = [all_genres, all_kws, all_production_companies, all_production_countries]
file_names = ['csv_for_db/genres_al.csv', 'csv_for_db/keywords_al.csv', 'csv_for_db/prod_company_al.csv', 'csv_for_db/prod_country_al.csv']
headers = [('genre_id', 'genre_name'), ('kw_id', 'kw_name'), ('prodcomp_id', 'prodcomp_name'), ('prodcountry_id', 'prodcountry_name')]

for d,f,h in zip(data_dicts, file_names, headers):
    print(f)
    fd = open(f, 'w')
    fd.write(",".join(h))
    for k,v in d.items():
        fd.write("\n" + str(k) + ',' + v)
    fd.close()

csv_for_db/genres_al.csv
csv_for_db/keywords_al.csv
csv_for_db/prod_company_al.csv
csv_for_db/prod_country_al.csv


In [27]:
pd.read_csv(r"csv_for_db/movie_details_al.csv")

,movieid,budget,original_language,title,overview,popularity,release_date,revenue,runtime,tagline,vote_average,vote_count
0,19995,237000000,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800
1,285,300000000,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500
2,206647,245000000,en,Spectre,A cryptic message from Bond's past sends him o...,107.376788,2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466
3,49026,250000000,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106
4,49529,260000000,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...
4798,9367,220000,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,2040920,81.0,"He didn't come looking for trouble, but troubl...",6.6,238
4799,72766,9000,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,0,85.0,A newlywed couple's honeymoon is upended by th...,5.9,5
4800,231617,0,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,0,120.0,NaN,7.0,6
4801,126186,0,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,0,98.0,A New Yorker in Shanghai,5.7,7


In [28]:
df_prod = pd.read_csv(r"csv_for_db/prod_company_al.csv")

In [29]:
df_prod[df_prod["prodcomp_id"] == 33]

,prodcomp_id,prodcomp_name
52,33,Universal Pictures


In [30]:
df.to_csv("csv_for_db/original_csv_al.csv")

In [31]:
df[["title", "production_companies"]]

,title,production_companies
0,Avatar,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,Pirates of the Caribbean: At World's End,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,Spectre,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,The Dark Knight Rises,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,John Carter,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"
...,...,...
4798,El Mariachi,"[{""name"": ""Columbia Pictures"", ""id"": 5}]"
4799,Newlyweds,[]
4800,"Signed, Sealed, Delivered","[{""name"": ""Front Street Pictures"", ""id"": 3958}..."
4801,Shanghai Calling,[]


In [56]:
g_vote= []

for row in df.iterrows():
    mg = eval(row[1]['genres'])
    movieid = row[1]['id']
    votecount = row[1]["vote_count"]
    
    for ele in mg:
        g_vote.append([ele['name'],ele['id'],votecount])

In [58]:
df_g_vote = pd.DataFrame(g_vote, columns=["g_name","g_id", "votecount"])

In [60]:
df_g_vote.to_csv("csv_for_db/genre_vote_csv_al.csv")

In [51]:
movie_genre_tuples[:2]

[(19995, 28), (19995, 12)]

In [61]:
df_g_vote

,g_name,g_id,votecount
0,Action,28,11800
1,Adventure,12,11800
2,Fantasy,14,11800
3,Science Fiction,878,11800
4,Adventure,12,4500
...,...,...,...
12155,Comedy,35,6
12156,Drama,18,6
12157,Romance,10749,6
12158,TV Movie,10770,6


In [63]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [65]:
df["overview"][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [67]:
p_avg_vote= []

for row in df.iterrows():
    mg = eval(row[1]['production_companies'])
    movieid = row[1]['id']
    avg_vote = row[1]['vote_average']
    
    for ele in mg:
        p_avg_vote.append([ele['id'],movieid, avg_vote])

In [68]:
df_p_avg_vote = pd.DataFrame(p_avg_vote, columns=["prod_id","movieid", "avg_vote"])

In [75]:
df_p_avg_vote

,prod_id,movieid,avg_vote
0,289,19995,7.2
1,306,19995,7.2
2,444,19995,7.2
3,574,19995,7.2
4,2,285,6.9
...,...,...,...
13672,5,9367,6.6
13673,3958,231617,7.0
13674,6438,231617,7.0
13675,87986,25975,6.3


In [154]:
df_p_avg_vote.to_csv("csv_for_db/pavgvote.csv")

In [85]:
df_p_avg_vote[df_p_avg_vote["prod_id"] == 289]

,prod_id,movieid,avg_vote
0,289,19995,7.2
84,289,2268,5.8
118,289,36668,6.3
384,289,49538,7.1
482,289,2080,6.2
741,289,6972,6.3
815,289,1979,5.4
822,289,87827,7.2
930,289,1593,6.3
1008,289,1571,6.4


In [122]:
# sirf 7 se jyada = (whole - 7 se kam - intersect of >7 and <7)

# A == >7
# B == <7

# A - A interesect B 
# #main dataset [df_p_avg_vote[df_p_avg_vote["avg_vote"]>7] ids  interesection df_p_avg_vote[df_p_avg_vote["avg_vote"]<7] ids ] 

,prod_id,movieid,avg_vote
0,289,19995,7.2
1,306,19995,7.2
2,444,19995,7.2
3,574,19995,7.2
4,2,285,6.9
...,...,...,...
13672,5,9367,6.6
13673,3958,231617,7.0
13674,6438,231617,7.0
13675,87986,25975,6.3


In [123]:
A = set(list(df_p_avg_vote[df_p_avg_vote["avg_vote"]>7]['prod_id'])) 
B = set(list(df_p_avg_vote[df_p_avg_vote["avg_vote"]<7]['prod_id']))

In [125]:
inter = A.intersection(B)

In [129]:
one = A.difference(B)
ans = one.difference(inter)

In [131]:
len(ans)

777

In [144]:
len(set(list(df_p_avg_vote["prod_id"])))

5047

In [132]:
ans

{46,
 49,
 54,
 77,
 82,
 86,
 88,
 98,
 100,
 120,
 125,
 156,
 157,
 179,
 182,
 194,
 195,
 200,
 209,
 210,
 218,
 219,
 223,
 224,
 234,
 245,
 250,
 260,
 266,
 268,
 269,
 270,
 271,
 286,
 294,
 312,
 313,
 315,
 336,
 337,
 358,
 361,
 367,
 373,
 379,
 381,
 382,
 388,
 394,
 395,
 398,
 399,
 406,
 413,
 416,
 418,
 462,
 463,
 464,
 465,
 469,
 472,
 473,
 474,
 486,
 501,
 503,
 504,
 517,
 518,
 523,
 524,
 526,
 541,
 542,
 543,
 549,
 554,
 560,
 571,
 592,
 631,
 633,
 636,
 638,
 646,
 654,
 690,
 692,
 741,
 766,
 778,
 793,
 794,
 795,
 798,
 851,
 865,
 881,
 893,
 921,
 922,
 924,
 938,
 999,
 1044,
 1095,
 1101,
 1102,
 1153,
 1157,
 1168,
 1182,
 1197,
 1232,
 1251,
 1297,
 1298,
 1307,
 1308,
 1315,
 1316,
 1317,
 1323,
 1324,
 1514,
 1515,
 1519,
 1533,
 1538,
 1553,
 1627,
 1656,
 1677,
 1687,
 1688,
 1689,
 1690,
 1691,
 1692,
 1779,
 1783,
 1819,
 1843,
 1848,
 1915,
 1916,
 1918,
 1997,
 2015,
 2019,
 2116,
 2264,
 2274,
 2278,
 2299,
 2307,
 2319,
 2325,


In [153]:
df_p_avg_vote[df_p_avg_vote["prod_id"]==11342]

,prod_id,movieid,avg_vote
10991,11342,8374,7.2


In [83]:
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [87]:
df_p_avg_vote["prod_id"].value_counts()

6194     319
33       311
4        285
306      222
5        201
        ... 
22872      1
14684      1
22880      1
12653      1
51199      1
Name: prod_id, Length: 5047, dtype: int64

In [120]:
df_1 = df.sort_values(by=['vote_count'], ascending=False)[["title","production_companies"]][:5]

In [121]:
for row in df_1.iterrows():
    mg = eval(row[1]['production_companies'])
    print(mg)

[{'name': 'Legendary Pictures', 'id': 923}, {'name': 'Warner Bros.', 'id': 6194}, {'name': 'Syncopy', 'id': 9996}]
[{'name': 'DC Comics', 'id': 429}, {'name': 'Legendary Pictures', 'id': 923}, {'name': 'Warner Bros.', 'id': 6194}, {'name': 'DC Entertainment', 'id': 9993}, {'name': 'Syncopy', 'id': 9996}]
[{'name': 'Ingenious Film Partners', 'id': 289}, {'name': 'Twentieth Century Fox Film Corporation', 'id': 306}, {'name': 'Dune Entertainment', 'id': 444}, {'name': 'Lightstorm Entertainment', 'id': 574}]
[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Marvel Studios', 'id': 420}]
[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}, {'name': 'Marvel Entertainment', 'id': 7505}, {'name': "The Donners' Company", 'id': 11307}, {'name': 'TSG Entertainment', 'id': 22213}, {'name': 'Kinberg Genre', 'id': 78091}]


In [92]:
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""